# Лабораторная 0

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

##### Работа с тензорами

In [8]:
# Создание тензоров
tensor_a = torch.tensor([[1.0, 2.0], 
                         [3.0, 4.0]])

tensor_b = torch.tensor([[5.0, 6.0], 
                         [7.0, 8.0]])

# Сложение тензоров
tensor_sum = tensor_a + tensor_b
print("\nСумма тензоров:\n", tensor_sum)

# Транспонирование тензора
tensor_transpose = tensor_a.T
print("\nТранспонирование тензора:\n", tensor_transpose)

# Матричное умножение тензоров
tensor_matmul = torch.matmul(tensor_a, tensor_b)
print("\nМатричное умножение тензоров:\n", tensor_matmul)


Сумма тензоров:
 tensor([[ 6.,  8.],
        [10., 12.]])

Транспонирование тензора:
 tensor([[1., 3.],
        [2., 4.]])

Матричное умножение тензоров:
 tensor([[19., 22.],
        [43., 50.]])


#### Градиенты 

Градиент для функции $ f(x) = x^2 $

1. Аналитическое вычисление градиента

    Для функции $ f(x) = x^2 $, аналитический градиент можно вычислить с помощью производной:
    $$
    f'(x) = \frac{d}{dx} (x^2) = 2x
    $$
    То есть, производная функции $ x^2 $ в точке $ x = a $ будет равна $ 2a $.

2. Вычисление градиента с PyTorch

    В PyTorch, используя автоматическое дифференцирование, можно вычислить градиент функции в произвольной точке, используя метод `backward()`.

In [55]:
x = torch.tensor(3.0, requires_grad=True)
y = x ** 2
y.backward()

# Печатаем значение градиента dy/dx в точке x=3
print(f"Градиент функции y=x^2 при x=3: {x.grad}")

Градиент функции y=x^2 при x=3: 6.0


#### Пояснение

Тензоры позволяют представлять входные данные (например, изображения, текст, аудио) и обучаемые параметры (например, веса нейронной сети) в формате матриц. Тензоры используются буквально везде в глубоком обучении, так как каждый шаг - это вычисление, выделение куска тензора и так далее.

Градиенты используются для оптимизации моделей. С помощью обратного распространения ошибки градиенты позволяют настраивать параметры модели, минимизируя функцию потерь.

# Лабораторная 1

## Часть 1: Введение в сверточные нейронные сети (CNN) для классификации изображений

Задачи:
1.	Загрузить датасет CIFAR-10, выполнить нормализацию и подготовить данные для обучения.
2.	Определить архитектуру сети с двумя сверточными слоями, пулингом и полносвязными слоями.
3.	Написать тренировочный цикл, запустить обучение и отслеживать точность на обучающей и тестовой выборках.
4.	Визуализировать функцию потерь и оценить точность, объяснив, какие улучшения можно внести для повышения производительности.

In [57]:
import torch
import torchvision
import torchvision.transforms as transforms

# Определяем трансформации для нормализации данных
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Нормализация
])

# Загружаем CIFAR-10 с помощью DataLoader
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [58]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # Первый сверточный слой
        self.pool = nn.MaxPool2d(2, 2)               # MaxPooling
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # Второй сверточный слой
        self.fc1 = nn.Linear(32 * 8 * 8, 120)        # Полносвязный слой
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)                 # Выходной слой для 10 классов

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))         # Сверточный слой + пулинг
        x = self.pool(F.relu(self.conv2(x)))         # Сверточный слой + пулинг
        x = x.view(-1, 32 * 8 * 8)                   # Вытягиваем тензор
        x = F.relu(self.fc1(x))                      # Полносвязный слой
        x = F.relu(self.fc2(x))
        x = self.fc3(x)                              # Выходной слой
        return x

net = SimpleCNN()

In [ ]:
import torch.optim as optim

# Определяем функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Тренировочный цикл
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()                   # Обнуляем градиенты
        outputs = net(inputs)                   # Прямой проход
        loss = criterion(outputs, labels)       # Вычисление функции потерь
        loss.backward()                         # Обратное распространение
        optimizer.step()                        # Шаг оптимизатора

        running_loss += loss.item()
        if i % 200 == 199:                      # Выводим каждые 200 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print("Обучение завершено.")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')

In [ ]:
import matplotlib.pyplot as plt

loss_history = []

# Обновляем тренировочный цикл, чтобы сохранять историю потерь
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 200 == 199:
            loss_history.append(running_loss / 200)
            running_loss = 0.0

# Визуализация функции потерь
plt.plot(loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss over Time')
plt.show()

## Часть 2: Классификация изображений с предобработкой и аугментацией данных

Задачи:
1.	Загрузить и предобработать CIFAR-10, Fashion-MNIST и SVHN, применяя различные техники аугментации.
2.	Обучить модель CNN на каждом из датасетов, сравнить точность и функцию потерь.
3.	Оценить влияние аугментации на результат и описать, какие методы лучше подходят для каждого датасета.
4.	Сделать выводы о том, как предобработка помогает улучшить обобщающие способности модели.


## Часть 3: Оптимизация архитектуры CNN

Задачи:
1.	Создать три варианта архитектуры CNN с разным количеством фильтров и размерами ядер.
2.	Добавить и настроить Batch Normalization и Dropout для предотвращения переобучения.
3.	Провести сравнение моделей по точности и функции потерь на тестовой выборке.
4.	Описать, какой подход лучше работает для данной задачи и почему, предлагая возможные улучшения.


## Часть 4: Оптимизация гиперпараметров с использованием нестандартных методов

Задачи:
1.	Обучить модель с циклическим и адаптивным темпами обучения, сравнить результаты.
2.	Протестировать разные значения batch size и настроить накопление градиентов для экономии памяти.
3.	Сравнить модели с разным количеством фильтров и слоев, чтобы оценить влияние каждого гиперпараметра.
4.	На основе полученных результатов предложить оптимальные настройки гиперпараметров для повышения точности и стабильности обучения.
